In [1]:
import os
import tensorflow as tf

In [2]:
base_dir = 'dogImages'
train_dir = os.path.join(base_dir,'train')
valid_dir = os.path.join(base_dir,'valid')
test_dir = os.path.join(base_dir,'test')

In [75]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation='relu'),
    #tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(133,activation = 'relu'),
    #tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(133, activation='softmax')
    
])

In [3]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [76]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=1e-4), metrics=['acc'])

In [4]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  
        batch_size=20,
        class_mode='categorical' )


validation_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',shuffle=False)


Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


In [23]:
from IPython.display import display 
from PIL import Image


In [77]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=334,  # 2000 images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=42,  # 1000 images = batch_size * steps
      verbose=1)

Epoch 1/20
334/334 [==============================] - 86s 257ms/step - loss: 4.8750 - acc: 0.0159 - val_loss: 4.8262 - val_acc: 0.0192
Epoch 2/20
334/334 [==============================] - 83s 249ms/step - loss: 4.7568 - acc: 0.0251 - val_loss: 4.6715 - val_acc: 0.0467
Epoch 3/20
334/334 [==============================] - 83s 248ms/step - loss: 4.6207 - acc: 0.0385 - val_loss: 4.5178 - val_acc: 0.0503
Epoch 4/20
334/334 [==============================] - 83s 248ms/step - loss: 4.5009 - acc: 0.0497 - val_loss: 4.4092 - val_acc: 0.0527
Epoch 5/20
334/334 [==============================] - 83s 249ms/step - loss: 4.4023 - acc: 0.0516 - val_loss: 4.2855 - val_acc: 0.0587
Epoch 6/20
334/334 [==============================] - 83s 248ms/step - loss: 4.3271 - acc: 0.0612 - val_loss: 4.2200 - val_acc: 0.0790
Epoch 7/20
334/334 [==============================] - 83s 248ms/step - loss: 4.2536 - acc: 0.0674 - val_loss: 4.1734 - val_acc: 0.0707
Epoch 8/20
334/334 [==============================] - 8

In [78]:
predict = model.evaluate_generator(test_generator)

In [79]:
print('loss = {}, acc = {}'.format(round(predict[0],2),round(predict[1],2)))

loss = 3.88, acc = 0.10999999940395355


In [52]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [21]:
import glob2
pix = glob2.glob("dogPredict/*")
pix


['dogPredict\\40.jpg',
 'dogPredict\\Alaskan_malamute_00318.jpg',
 'dogPredict\\Australian_terrier_00924.jpg',
 'dogPredict\\osama.png',
 'dogPredict\\tiger.jpg']

In [22]:
from tensorflow.keras.preprocessing import image                  
# https://medium.com/@imBharatMishra/dog-breed-classification-with-keras-848b9b1525c1

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)



In [111]:
img = path_to_tensor('dogPredict\\40.jpg')

In [112]:
np.argmax(model.predict(img))

56

In [23]:
dog_names = [item[20:-1] for item in sorted(glob2.glob("dogImages/train/*/"))]


In [24]:
len(dog_names)

133

In [25]:
dog_names_dict ={ i : dog_names[i] for i in range(0, len(dog_names) ) }


In [124]:
def prediction(img_path):
    img = path_to_tensor(img_path)
    predict = np.argmax(model.predict(img))
    return dog_names_dict.get(predict)

In [125]:
prediction('dogPredict\\40.jpg')

'Dalmatian'

In [130]:
prediction('dogPredict\\tiger.jpg')

'Border_terrier'

In [138]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 86528)            

In [141]:
model.save('cnn.h5') 


In [142]:
cnn_model = tf.keras.models.load_model('cnn.h5')


In [143]:
def prediction(img_path):
    img = path_to_tensor(img_path)
    predict = np.argmax(cnn_model.predict(img))
    return dog_names_dict.get(predict)

In [145]:
prediction('dogPredict\\tiger.jpg')

'Border_terrier'

#### Transfer learning

In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [8]:
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (224, 224, 3), 
                                include_top = False, # dont call classifier part only feature extraction part
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
    layer.trainable = False # don't retrain the feature extraction weights 


In [9]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 12, 12, 768)


In [10]:

# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense  (133,activation = 'softmax')(x)           

model = tf.keras.Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])


In [11]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [13]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [14]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=334,  # 2000 images = batch_size * steps
      epochs=20,
      validation_data=validation_generator,
      validation_steps=42,  # 1000 images = batch_size * steps
      verbose=1)

Epoch 1/20
334/334 [==============================] - 102s 306ms/step - loss: 3.7404 - acc: 0.1623 - val_loss: 2.4565 - val_acc: 0.3713
Epoch 2/20
334/334 [==============================] - 95s 284ms/step - loss: 2.8892 - acc: 0.2802 - val_loss: 1.8829 - val_acc: 0.4958
Epoch 3/20
334/334 [==============================] - 96s 288ms/step - loss: 2.4154 - acc: 0.3728 - val_loss: 1.5866 - val_acc: 0.5401
Epoch 4/20
334/334 [==============================] - 100s 301ms/step - loss: 2.1100 - acc: 0.4302 - val_loss: 1.3740 - val_acc: 0.5880
Epoch 5/20
334/334 [==============================] - 98s 295ms/step - loss: 1.8937 - acc: 0.4787 - val_loss: 1.2497 - val_acc: 0.6323
Epoch 6/20
334/334 [==============================] - 95s 284ms/step - loss: 1.7430 - acc: 0.5162 - val_loss: 1.2578 - val_acc: 0.6228
Epoch 7/20
334/334 [==============================] - 95s 285ms/step - loss: 1.6255 - acc: 0.5386 - val_loss: 1.2145 - val_acc: 0.6455
Epoch 8/20
334/334 [==============================] -

In [15]:
predict = model.evaluate_generator(test_generator)

In [16]:
print('loss = {}, acc = {}'.format(round(predict[0],2),round(predict[1],2)))

loss = 1.21, acc = 0.7200000286102295


In [17]:
def prediction(img_path):
    img = path_to_tensor(img_path)
    predict = np.argmax(model.predict(img))
    return dog_names_dict.get(predict)

In [19]:
import numpy as np

In [26]:
prediction('dogPredict\\tiger.jpg')

'Chihuahua'

In [27]:
model.save('cnn_trans.h5') 
